In [2]:
import uuid
import logging
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct
from sentence_transformers import SentenceTransformer

# Параметры
COLL_NAME = 'documents'
ENCODER_NAME = 'intfloat/multilingual-e5-large'
VEC_SIZE = 1024  # Обновлено для соответствия размеру векторов энкодера
CHUNK_SIZE = 1430
CHUNK_OVERLAP = 138
FILES_DIR = 'files'

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levellevelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Функция для чтения файла и разбиения его на чанки
def file_to_chunks(file_path, chunk_size, chunk_overlap):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    chunks = []
    for i in range(0, len(text), chunk_size - chunk_overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

# Функция для преобразования текста в вектора
def str_to_vec(encoder, texts):
    return encoder.encode(texts)

# Создаем подключение к векторной БД
try:
    qdrant_client = QdrantClient('localhost', port=6333)
    logger.info("Successfully connected to Qdrant server.")
except Exception as e:
    logger.error(f"Failed to connect to Qdrant server: {e}")
    raise

# Функция для сохранения чанков в базу данных
def save_chunks(encoder, chunks, file_name):
    try:
        chunk_embeddings = str_to_vec(encoder, chunks)
        points = []
        for i in range(len(chunk_embeddings)):
            point = PointStruct(
                id=str(uuid.uuid4()),
                vector=chunk_embeddings[i],
                payload={'file': file_name, 'chunk': chunks[i], 'file_path': os.path.join(FILES_DIR, file_name)}
            )
            points.append(point)
        operation_info = qdrant_client.upsert(
            collection_name=COLL_NAME,
            wait=True,
            points=points
        )
        logger.info(f"Successfully saved chunks for file {file_name}.")
        return operation_info
    except Exception as e:
        logger.error(f"Failed to save chunks for file {file_name}: {e}")
        raise

# Функция для загрузки файлов в базу данных
def files_to_vecdb(files, encoder, vec_size, chunk_size, chunk_overlap):
    try:
        qdrant_client.delete_collection(collection_name=COLL_NAME)
        logger.info(f"Collection {COLL_NAME} deleted successfully.")
        qdrant_client.create_collection(
            collection_name=COLL_NAME,
            vectors_config=VectorParams(size=vec_size, distance=Distance.COSINE),
        )
        logger.info(f"Collection {COLL_NAME} created successfully.")
        for file_name in files:
            chunks = file_to_chunks(os.path.join(FILES_DIR, file_name), chunk_size, chunk_overlap)
            save_chunks(encoder, chunks, file_name)
    except Exception as e:
        logger.error(f"Failed to process files to vecdb: {e}")
        raise

# Инициализация энкодера
encoder_model = SentenceTransformer(ENCODER_NAME)

# Загрузка файлов в базу данных
import os
files = [f for f in os.listdir(FILES_DIR) if os.path.isfile(os.path.join(FILES_DIR, f))]
files_to_vecdb(files, encoder_model, VEC_SIZE, CHUNK_SIZE, CHUNK_OVERLAP)

2024-05-31 16:03:21,788 - __main__ - INFO - Successfully connected to Qdrant server.
2024-05-31 16:03:21,789 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-large
2024-05-31 16:03:25,008 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cuda
2024-05-31 16:03:26,020 - httpx - INFO - HTTP Request: DELETE http://localhost:6333/collections/documents "HTTP/1.1 200 OK"
2024-05-31 16:03:26,021 - __main__ - INFO - Collection documents deleted successfully.
2024-05-31 16:03:26,221 - httpx - INFO - HTTP Request: PUT http://localhost:6333/collections/documents "HTTP/1.1 200 OK"
2024-05-31 16:03:26,222 - __main__ - INFO - Collection documents created successfully.


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2024-05-31 16:03:28,341 - httpx - INFO - HTTP Request: PUT http://localhost:6333/collections/documents/points?wait=true "HTTP/1.1 200 OK"
2024-05-31 16:03:28,343 - __main__ - INFO - Successfully saved chunks for file 10669.txt.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-05-31 16:03:29,811 - httpx - INFO - HTTP Request: PUT http://localhost:6333/collections/documents/points?wait=true "HTTP/1.1 200 OK"
2024-05-31 16:03:29,813 - __main__ - INFO - Successfully saved chunks for file 106717.txt.


...

2024-05-31 16:04:17,631 - httpx - INFO - HTTP Request: PUT http://localhost:6333/collections/documents/points?wait=true "HTTP/1.1 200 OK"
2024-05-31 16:04:17,632 - __main__ - INFO - Successfully saved chunks for file 86619.txt.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-05-31 16:04:17,967 - httpx - INFO - HTTP Request: PUT http://localhost:6333/collections/documents/points?wait=true "HTTP/1.1 200 OK"
2024-05-31 16:04:17,968 - __main__ - INFO - Successfully saved chunks for file 92937.txt.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-05-31 16:04:18,124 - httpx - INFO - HTTP Request: PUT http://localhost:6333/collections/documents/points?wait=true "HTTP/1.1 200 OK"
2024-05-31 16:04:18,125 - __main__ - INFO - Successfully saved chunks for file 984291.txt.


In [6]:
# Вывод количества уникальных файлов в базе данных
try:
    count_result = qdrant_client.count(collection_name=COLL_NAME)
    print(f"Number of points in collection: {count_result.count}")

    all_points = qdrant_client.scroll(collection_name=COLL_NAME, limit=count_result.count, with_payload=True)
    file_names = set()
    for point in all_points.points:
        file_names.add(point.payload['file'])
    print("Number of unique files in collection:", len(file_names))
except Exception as e:
    logger.error(f"Failed to retrieve file count from collection {COLL_NAME}: {e}")

2024-05-31 16:06:48,568 - httpx - INFO - HTTP Request: POST http://localhost:6333/collections/documents/points/count "HTTP/1.1 200 OK"
2024-05-31 16:06:48,599 - httpx - INFO - HTTP Request: POST http://localhost:6333/collections/documents/points/scroll "HTTP/1.1 200 OK"
2024-05-31 16:06:48,676 - __main__ - ERROR - Failed to retrieve file count from collection documents: 'tuple' object has no attribute 'points'


Number of points in collection: 1093
